In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e6/sample_submission.csv
/kaggle/input/playground-series-s4e6/train.csv
/kaggle/input/playground-series-s4e6/test.csv


In [17]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [18]:
import numpy as np
import pandas as pd
# Import Libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier

In [19]:
traindata=pd.read_csv("/kaggle/input/playground-series-s4e6/train.csv")
testdata=pd.read_csv("/kaggle/input/playground-series-s4e6/test.csv")
traindata.head(5)

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0,1,1,1,9238,1,1,126.0,1,1,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,Graduate
1,1,1,17,1,9238,1,1,125.0,1,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout
2,2,1,17,2,9254,1,1,137.0,1,3,...,0,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout
3,3,1,1,3,9500,1,1,131.0,1,19,...,0,8,11,7,12.820000,0,11.1,0.6,2.02,Enrolled
4,4,1,1,2,9500,1,1,132.0,1,19,...,0,7,12,6,12.933333,0,7.6,2.6,0.32,Graduate


In [20]:
traindata.isna().sum()

id                                                0
Marital status                                    0
Application mode                                  0
Application order                                 0
Course                                            0
Daytime/evening attendance                        0
Previous qualification                            0
Previous qualification (grade)                    0
Nacionality                                       0
Mother's qualification                            0
Father's qualification                            0
Mother's occupation                               0
Father's occupation                               0
Admission grade                                   0
Displaced                                         0
Educational special needs                         0
Debtor                                            0
Tuition fees up to date                           0
Gender                                            0
Scholarship 

In [21]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
traindata['Target'] = label_encoder.fit_transform(traindata['Target'])


In [22]:
traindata.head(5)

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0,1,1,1,9238,1,1,126.0,1,1,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,2
1,1,1,17,1,9238,1,1,125.0,1,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,0
2,2,1,17,2,9254,1,1,137.0,1,3,...,0,6,0,0,0.000000,0,16.2,0.3,-0.92,0
3,3,1,1,3,9500,1,1,131.0,1,19,...,0,8,11,7,12.820000,0,11.1,0.6,2.02,1
4,4,1,1,2,9500,1,1,132.0,1,19,...,0,7,12,6,12.933333,0,7.6,2.6,0.32,2


In [23]:
X=traindata.drop(['Target'],axis=1)
y=traindata['Target']

In [24]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)


In [25]:

RandomForestClassifierModel = RandomForestClassifier(criterion = 'gini',n_estimators=100,max_depth=2,random_state=33) #criterion can be also : entropy
RandomForestClassifierModel.fit(x_train, y_train)
#Calculating Prediction
RFC_y_pred = RandomForestClassifierModel.predict(x_test)
# Calculate accuracy
val_accuracy = accuracy_score(y_test, RFC_y_pred)
print(f"Accuracy Score: {val_accuracy}")

# Calculate precision
precision = precision_score(y_test, RFC_y_pred, average='macro') 
print(f"Precision: {precision}")

# Calculate recall
recall = recall_score(y_test, RFC_y_pred, average='macro')
print(f"Recall: {recall}")

# Calculate F1-score
f1 = f1_score(y_test, RFC_y_pred, average='macro') 
print(f"F1-Score: {f1}")

Accuracy Score: 0.7422242550967068
Precision: 0.8274990936720177
Recall: 0.6106289467536007
F1-Score: 0.546148541037842


In [26]:


DecisionTreeClassifierModel = DecisionTreeClassifier(criterion='gini',max_depth=3,random_state=33) #criterion can be entropy
DecisionTreeClassifierModel.fit(x_train, y_train)

#Calculating Prediction
DTC_y_pred = DecisionTreeClassifierModel.predict(x_test)
# Calculate accuracy
val_accuracy = accuracy_score(y_test, DTC_y_pred)
print(f"Accuracy Score: {val_accuracy}")

# Calculate precision
precision = precision_score(y_test, DTC_y_pred, average='macro') 
print(f"Precision: {precision}")

# Calculate recall
recall = recall_score(y_test, DTC_y_pred, average='macro') 
print(f"Recall: {recall}")

# Calculate F1-score
f1 = f1_score(y_test, DTC_y_pred, average='macro')
print(f"F1-Score: {f1}")

Accuracy Score: 0.8012284370099321
Precision: 0.77294918185213
Recall: 0.7470143980347269
F1-Score: 0.7564769205994676


In [27]:

KNNClassifierModel = KNeighborsClassifier(n_neighbors= 5,weights ='uniform', # it can be distance
                                          algorithm='auto') # it can be ball_tree, kd_tree,brute
KNNClassifierModel.fit(x_train, y_train)

#Calculating Prediction
KNN_y_pred = KNNClassifierModel.predict(x_test)
# Calculate accuracy
val_accuracy = accuracy_score(y_test, KNN_y_pred)
print(f"Accuracy Score: {val_accuracy}")
# Calculate precision
precision = precision_score(y_test, KNN_y_pred, average='macro')
print(f"Precision: {precision}")

# Calculate recall
recall = recall_score(y_test, KNN_y_pred, average='macro') 
print(f"Recall: {recall}")

# Calculate F1-score
f1 = f1_score(y_test, KNN_y_pred, average='macro') 
print(f"F1-Score: {f1}")

Accuracy Score: 0.5222164140094093
Precision: 0.464349254841403
Recall: 0.4637644694563634
F1-Score: 0.4628067195813792


In [28]:
# Initializing CatBoostClassifier
CatBoostClassifierModel = CatBoostClassifier(iterations=100, 
                                             depth=2, 
                                             learning_rate=0.1, 
                                             random_seed=33,
                                             logging_level='Silent')

# Fitting the model
CatBoostClassifierModel.fit(x_train, y_train)

# Calculating Prediction
CatBoost_y_pred = CatBoostClassifierModel.predict(x_test)

# Calculate accuracy
val_accuracy = accuracy_score(y_test, CatBoost_y_pred)
print(f"Accuracy Score: {val_accuracy}")

# Calculate precision
precision = precision_score(y_test, CatBoost_y_pred, average='macro') 
print(f"Precision: {precision}")

# Calculate recall
recall = recall_score(y_test, CatBoost_y_pred, average='macro')  
print(f"Recall: {recall}")

# Calculate F1-score
f1 = f1_score(y_test, CatBoost_y_pred, average='macro')  
print(f"F1-Score: {f1}")

Accuracy Score: 0.8181521170935703
Precision: 0.7875295387325444
Recall: 0.7683149196139798
F1-Score: 0.7759642337610565


In [29]:
testdata.head(5)

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
0,76518,1,1,1,9500,1,1,141.0,1,3,...,0,0,8,0,0,0.000000,0,13.9,-0.3,0.79
1,76519,1,1,1,9238,1,1,128.0,1,1,...,0,0,6,6,6,13.500000,0,11.1,0.6,2.02
2,76520,1,1,1,9238,1,1,118.0,1,1,...,0,0,6,11,5,11.000000,0,15.5,2.8,-4.06
3,76521,1,44,1,9147,1,39,130.0,1,1,...,0,3,8,14,5,11.000000,0,8.9,1.4,3.51
4,76522,1,39,1,9670,1,1,110.0,1,1,...,0,0,6,9,4,10.666667,2,7.6,2.6,0.32


In [33]:
test_predictions = CatBoostClassifierModel.predict(testdata.drop(['id'], axis=1))


In [31]:
print(test_predictions.shape)

(51012, 1)


In [34]:
test_predictions = CatBoostClassifierModel.predict(testdata.drop(['id'], axis=1)).flatten()

# Now create the submission DataFrame
submission = pd.DataFrame({'id': testdata['id'], 'Target': test_predictions})

# Save to CSV
submission.to_csv('submission.csv', index=False)
print("\nSubmission file created successfully!")


Submission file created successfully!
